<a href="https://colab.research.google.com/github/ghoshatanu857/Instrument_Automation/blob/main/Vector_Magnetometry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from plotly.subplots import make_subplots
from plotly import graph_objs as go
import plotly.figure_factory as ff

In [2]:
fig_template = go.layout.Template()
fig_template.layout = {
    'template': 'simple_white+presentation',
    'autosize': False,
    'width': 800,
    'height': 600,
    # 'opacity': 0.2,
    'xaxis': {
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 1.5+0.5,
        'tickwidth': 1.5+0.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white',
        },
    'yaxis': {
        'ticks': 'inside',
        'mirror': 'ticks',
        'linewidth': 1.5+0.5,
        'tickwidth': 1.5+0.5,
        'ticklen': 6,
        'showline': True,
        'showgrid': False,
        'zerolinecolor': 'white'
        },
    'font':{'family':'mathjax',
            'size': 22,
            }
}

In [3]:
def magField(current,distance,xy_points):
  if type(xy_points)!=np.ndarray:
    field = (4*np.pi*1e-7*current)/(2*np.pi*distance)
  else:
    field = np.ones((xy_points.shape[1],xy_points.shape[2]))
    for i in range(xy_points.shape[1]):
      for j in range(xy_points.shape[2]):
        x=xy_points[:,i,j]
        if (x[0]==0 and x[1]==0):
          field[i,j] = None
        else:
          field[i,j] = (4*np.pi*1e-7*current)/(2*np.pi*np.sqrt(x[0]**2+x[1]**2))
  return field

In [4]:
def circle(inner_r,outer_r,circle_points,line_points):
  x_array=[]; y_array=[]
  x_array.append([points*np.cos(_points) for points,_points in zip(line_points,circle_points)])
  y_array.append([points*np.sin(_points) for points,_points in zip(line_points,circle_points)])
  return x_array, y_array

In [26]:
wire_radius = 25e-6; distance = 100e-6; wire_current = 10
linear_density = 50; init_angle = 0; final_angle = 2*np.pi
min_circular_density = 10; point_addition = 1

linear_points = np.linspace(wire_radius, distance, linear_density); inside_points = np.zeros((10))
circular_points=[]; circular_angles=[]
[circular_points.append(int(min_circular_density + circular_density*point_addition)) for circular_density in range(linear_density)]
circular_points = np.array(circular_points)
if np.all(circular_points)==False:
  raise Exception('Check the values of circular_points')
circular_angles = [np.linspace(init_angle, final_angle, _points, endpoint=False) for _points in circular_points]

mag_field = magField(current=wire_current, xy_points=False,distance=linear_points)
x_points, y_points = circle(wire_radius, distance, circular_angles, line_points=linear_points)

In [6]:
circular_points

array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43,
       44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59])

In [104]:
Bnew = magField(current=wire_current,xy_points=False, distance=np.concatenate((inside_points,linear_points)))
Bnew[np.where(np.isinf(Bnew))[0]]=0
Bnew_x=[]; Bnew_y=[]
[Bnew_x.append(b*np.cos(np.linspace(init_angle,final_angle,100))) for b in Bnew]
[Bnew_y.append(b*np.sin(np.linspace(init_angle,final_angle,100))) for b in Bnew]
Bnew_x = np.array(Bnew_x); Bnew_y = np.array(Bnew_y)
Bnew_z = np.sqrt(Bnew_x**2 + Bnew_y**2)
Bnew_z[np.where(Bnew_z==0)[0]]=None

<ipython-input-3-d5b071b0d85f>:3: RuntimeWarning:

divide by zero encountered in divide



In [163]:
fig = go.Figure()
[fig.add_trace(go.Scatter(x=B_x, y=B_y, mode='markers',marker=dict(color = B_z[0]))) for B_x,B_y,B_z in zip(Bnew_x,Bnew_y,Bnew_z)]

fig.update_layout(showlegend=False)
fig.update_layout(template=fig_template, title = 'Magnetic Field ',width=800, height=600)
fig.show()

In [100]:
linear_point = np.concatenate((-linear_points[::-1],inside_points,linear_points))
# linear_point = np.concatenate((-linear_points[::-1],linear_points))

X, Y = np.meshgrid(linear_point, linear_point)
radius = np.sqrt(X**2 + Y**2)

# B = magField(current=wire_current, distance=radius, xy_points=False)
B = magField(current=wire_current, distance=radius, xy_points=np.array([X,Y]))

In [169]:
fig = go.Figure(data=go.Contour(x=linear_point,y=linear_point,z=B,contours_coloring='heatmap',
                                connectgaps = True,line_smoothing=1,colorbar=dict(title='Magnetic Field',titleside='right'),colorscale=None))
fig.update_traces(line_width=0)
fig.add_trace(go.Scatter(x=wire_radius*np.cos(np.linspace(0,2*np.pi,100)), y=wire_radius*np.sin(np.linspace(0,2*np.pi,100)),
                         mode='lines', line=dict(color='yellow', width=0.5),fill = 'tonexty',fillcolor = 'rgba(135, 206, 250, 0.3)'))

for radi in range(0,len(x_points[0]),8):
    x = x_points[0][radi]; y = y_points[0][radi]
    u = -y; v = x
    fig1 = ff.create_quiver(x, y, u, v, scale=125e-6*mag_field[radi]/linear_points[radi], arrow_scale=0.5, scaleratio=1.0, angle=np.pi/9, line=dict(width=1.2, color='black'))
    fig.add_traces(data=fig1.data)

fig.add_trace(go.Scatter(x=[0], y=[0], mode='markers', marker_size=15, marker=dict(color='yellow')))
fig.add_trace(go.Scatter(x=linear_point, y=-wire_radius*np.ones(linear_point.shape[0]), mode='lines', marker=dict(color='yellow'),fill=None))
fig.add_trace(go.Scatter(x=linear_point, y=-distance*np.ones(linear_point.shape[0]), mode='lines', marker=dict(color='grey'),
                         fill='tonexty',fillcolor = 'rgba(255, 255, 255, 0.4)'))

fig.update_layout(showlegend=False)
fig.update_layout(template=fig_template, title = 'Magnetic Field vs Radial Distance',width=800, height=600)
fig.update_xaxes(range = [-distance,distance]); fig.update_yaxes(range = [-distance,distance])
fig.show()